# High Frequency Trading Algorithm

You have been tasked by the investment firm Renaissance High Frequency Trading (RHFT) to develop an automated trading strategy utilizing a combination of machine learning algorithms and high frequency algorithms. RHFT wants this new algorithm to be based on stock market data of the 30 stocks in the Dow Jones at the minute level and to conduct buys and sells every minute based on 1 min, 5 min, and 10 min Momentum. The CIO asked you to choose the Machine Learning Algorithm best suited for this task and wants you to execute the trades via Alpaca's API.

### Initial Set-Up

In [2]:
import os
from pathlib import Path
import alpaca_trade_api as tradeapi
import pandas as pd
import numpy as np
import datetime
import time
from dotenv import load_dotenv

In [1]:
!pip install alpaca_trade_api

  Using cached alpaca_trade_api-1.1.0-py3-none-any.whl (39 kB)


In [3]:
# Load .env enviroment variables
# YOUR CODE HERE
load_dotenv('key.env')

True

In [4]:
# Set Alpaca API key and secret
# YOUR CODE HERE
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [5]:
# Create the Alpaca API object, specifying use of the paper trading account:
# YOUR CODE HERE
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

## Part 2: Train and Compare Multiple Machine Learning Algorithms

 In this section, you'll train each of the requested algorithms and compare performance. Be sure to use the same parameters and training steps for each model. This is necessary to compare each model accurately.

### Preprocessing Data

#### 1. Generate your feature data (`X`) and target data (`y`):
* Create a dataframe `X` that contains all the columns from the returns dataframe that will be used to predict `F_1_m_returns`.
* Create a variable, called `y`, that is equal 1 if `F_1_m_returns` is larger than 0. This will be our target variable.

In [8]:
# Load the dataset returns.csv and set the index to level_0 and time
# YOUR CODE HERE
file_path = Path('returns.csv')
df = pd.read_csv(file_path)

# Preview the data
pd.DataFrame(df)

,level_0,level_1,F_1_m_returns,1_m_returns,5_m_returns,10_m_returns
0,FB,2021-01-05 09:40:00-05:00,0.000814,0.000074,0.001260,0.004758
1,FB,2021-01-05 09:41:00-05:00,0.000887,0.000814,0.001889,0.004941
2,FB,2021-01-05 09:42:00-05:00,0.000628,0.000887,0.001999,0.003782
3,FB,2021-01-05 09:43:00-05:00,0.000480,0.000628,0.003408,0.007850
4,FB,2021-01-05 09:44:00-05:00,-0.001291,0.000480,0.002886,0.005416
5,FB,2021-01-05 09:45:00-05:00,-0.000961,-0.001291,0.001517,0.002778
6,FB,2021-01-05 09:46:00-05:00,-0.000444,-0.000961,-0.000259,0.001630
7,FB,2021-01-05 09:47:00-05:00,0.001961,-0.000444,-0.001588,0.000407
8,FB,2021-01-05 09:48:00-05:00,0.000480,0.001961,-0.000258,0.003148
9,FB,2021-01-05 09:49:00-05:00,0.001993,0.000480,-0.000258,0.002627


In [18]:
# Create a separate dataframe for features and define the target variable as a binary target
# YOUR CODE HERE

# Create the target variable
# YOUR CODE HERE

##### Note:
> Notice that we don't use shuffle when splitting the dataset into a training and testing dataset. 

> We want to keep the original ordering of the data, so we don't end up using observations in the future to predict past observations,

> This is a critical mistake known as look ahead bias.

#### 2. Use the train_test_split library to split the dataset into a training and testing dataset, with 70% used for testing
* Set the shuffle parameter to False, so that you use the first 70% for training to prvent look ahead bias.
* Make sure you have these 4 variables: `X_train`, `X_test`, `y_train`, `y_test`. 

In [19]:
# Import train_test_split 
from sklearn.model_selection import train_test_split

# Split the dataset without shuffling
# YOUR CODE HERE

#### 3. Use the `Counter` function to test the distribution of the data. 
* The result of `Counter({1: 668, 0: 1194})` reveals the data is indeed unbalanced.

In [20]:
# Import the Counter function from the collections library
from collections import Counter

# Use Counter to count the number 1s and 0 in y_train
# YOUR CODE HERE

Counter({1: 668, 0: 1194})

#### 4. Balance the dataset with the Oversampler libary, setting `random state= 1`.

In [21]:
# Import RandomOverSampler from the imblearn library
from imblearn.over_sampling import RandomOverSampler

# Use RandomOverSampler to resample the datase using random_state=1
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

#### 5. Test the distribution once again with `Counter`. The new result of `Counter({1: 1194, 0: 1194})` shows the data is now balanced.

In [22]:
# Use Counter again to verify imbalance removed
# YOUR CODE HERE

Counter({1: 1194, 0: 1194})

# Machine Learning

#### 1. The first cells in this section provide an example of how to fit and train your model using the `LogisticRegression` model from sklearn:
* Import select model.
* Instantiate model object.
* Fit the model to the resampled data - `X_resampled` and `y_resampled`.
* Predict the model using `X_test`.
* Print the classification report.

In [23]:
# Import classification_report from sklearn
from sklearn.metrics import classification_report

In [24]:
# Import LogisticRegression from sklearn
from sklearn.linear_model import LogisticRegression

# Create a LogisticRegression model and train it on the X_resampled data we created before
log_model = LogisticRegression()
log_model.fit(X_resampled, y_resampled)  

# Use the model you trained to predict using X_test
y_pred = log_model.predict(X_test)   

# Print out a classification report toevaluate performance
print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.5365    0.5259    0.5312       405
           1     0.5212    0.5318    0.5264       393

    accuracy                         0.5288       798
   macro avg     0.5289    0.5289    0.5288       798
weighted avg     0.5290    0.5288    0.5288       798



#### 2. Use the same approach as above to train and test the following ML Algorithms:
* [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
* [GradientBoostingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html)
* [AdaBoostClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)
* [XGBClassifier](https://xgboost.readthedocs.io/en/latest/python/python_api.html)

#### RandomForestClassifier

In [25]:
# Import RandomForestClassifier from sklearn
from sklearn.ensemble import RandomForestClassifier

# Create a RandomForestClassifier model and train it on the X_resampled data we created before
# YOUR CODE HERE

# Use the model you trained to predict using X_test
# YOUR CODE HERE  

# Print out a classification report to evaluate performance
# YOUR CODE HERE

              precision    recall  f1-score   support

           0     0.5041    0.6049    0.5499       405
           1     0.4872    0.3868    0.4312       393

    accuracy                         0.4975       798
   macro avg     0.4956    0.4959    0.4906       798
weighted avg     0.4958    0.4975    0.4915       798



#### GradientBoostingClassifier

In [26]:
# Import RandomForestClassifier from sklearn
from sklearn.ensemble import GradientBoostingClassifier

# Create a GradientBoostingClassifier model and train it on the X_resampled data we created before
# YOUR CODE HERE

# Use the model you trained to predict using X_test
# YOUR CODE HERE     

# Print out a classification report to evaluate performance
# YOUR CODE HERE

              precision    recall  f1-score   support

           0     0.4976    0.5136    0.5055       405
           1     0.4816    0.4656    0.4735       393

    accuracy                         0.4900       798
   macro avg     0.4896    0.4896    0.4895       798
weighted avg     0.4897    0.4900    0.4897       798



#### AdaBoostClassifier

In [27]:
# Import RandomForestClassifier from sklearn
from sklearn.ensemble import AdaBoostClassifier

# Create a AdaBoostClassifier model and train it on the X_resampled data we created before
# YOUR CODE HERE

# Use the model you trained to predict using X_test
# YOUR CODE HERE

# Print out a classification report to evaluate performance
# YOUR CODE HERE

              precision    recall  f1-score   support

           0     0.4951    0.3753    0.4270       405
           1     0.4847    0.6056    0.5385       393

    accuracy                         0.4887       798
   macro avg     0.4899    0.4905    0.4827       798
weighted avg     0.4900    0.4887    0.4819       798



#### XGBClassifier

In [28]:
# Import RandomForestClassifier from sklearn
from xgboost import XGBClassifier

# Create a XGBClassifier model and train it on the X_resampled data we created before
# YOUR CODE HERE

# Use the model you trained to predict using X_test
# YOUR CODE HERE

# Print out a classification report to evaluate performance
# YOUR CODE HERE

C:\Users\lorie\Anaconda3\envs\pyvizenv25\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:23:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0     0.4901    0.5481    0.5175       405
           1     0.4696    0.4122    0.4390       393

    accuracy                         0.4812       798
   macro avg     0.4798    0.4802    0.4783       798
weighted avg     0.4800    0.4812    0.4788       798



### Evaluate the performance of each model


#### 1. Using the classification report for each model, choose the model with the highest precision for use in your algo-trading program.
#### 2. Save the selected model with the `joblib` libary to avoid retraining every time you wish to use it.

In [29]:
# Import the joblib library 
import joblib

# Use the library to save the model that you want to use for trading
joblib.dump(log_model, 'log_model.pkl')

['log_model.pkl']

## Part 3: Implement the strongest model using Apaca API

### Develop the Algorithm


#### 1. Use the provided code to ping the Alpaca API and create the DataFrame needed to feed data into the model.
   * This code will also store the correct feature data in `X` for later use.

In [30]:
# Create the list of tickers

ticker_list = ['FB','AMZN','AAPL','NFLX', 'GOOGL', 'MSFT', 'TSLA']
# Define Dates

beg_date = '2021-01-06'
end_date = '2021-01-06'

# Convert the date in a format the Alpaca API reqires
start =  pd.Timestamp(f'{beg_date} 09:30:00-0400', tz='America/New_York').replace(hour=9, minute=30, second=0).astimezone('GMT').isoformat()[:-6]+'Z'
end   =  pd.Timestamp(f'{end_date} 16:00:00-0400', tz='America/New_York').replace(hour=15, minute=0, second=0).astimezone('GMT').isoformat()[:-6]+'Z'
timeframe='1Min'

# Use iloc to get the last 10 mins every time we pull new data
prices = api.get_barset(ticker_list, "minute", start=start, end=end).df.iloc[-11:]
prices.ffill(inplace=True)   

# Create an empty DataFrame for closing prices
df_closing_prices = pd.DataFrame()

# Fetch the closing prices of our tickers
df_closing_prices["FB"] = prices["FB"]["close"]
df_closing_prices["AMZN"] = prices["AMZN"]["close"]
df_closing_prices["AAPL"] = prices["AAPL"]["close"]
df_closing_prices["NFLX"] = prices["NFLX"]["close"]
df_closing_prices["GOOGL"] = prices["GOOGL"]["close"]
df_closing_prices['MSFT'] = prices['MSFT']["close"]
df_closing_prices['TSLA'] = prices['TSLA']["close"]

print(df_closing_prices.head(20))

                                FB      AMZN     AAPL    NFLX    GOOGL  \
time                                                                     
2021-01-06 14:50:00-05:00  264.610  3146.960  127.110  506.54  1721.82   
2021-01-06 14:51:00-05:00  264.630  3146.910  127.430  506.54  1721.82   
2021-01-06 14:52:00-05:00  264.830  3147.980  127.720  506.69  1723.67   
2021-01-06 14:53:00-05:00  264.525  3148.570  127.510  506.01  1723.67   
2021-01-06 14:54:00-05:00  264.560  3147.840  127.645  506.01  1720.84   
2021-01-06 14:55:00-05:00  264.880  3150.330  127.920  506.30  1720.60   
2021-01-06 14:56:00-05:00  264.965  3150.610  128.150  506.72  1721.10   
2021-01-06 14:57:00-05:00  264.980  3151.745  127.980  507.07  1720.07   
2021-01-06 14:58:00-05:00  265.000  3149.280  127.850  506.33  1720.07   
2021-01-06 14:59:00-05:00  265.360  3150.840  127.930  506.13  1720.48   
2021-01-06 15:00:00-05:00  264.840  3148.580  127.630  506.43  1720.48   

                              MSFT   

In [31]:
# Create list of momentums
list_of_momentums = [1,5,10]

for i in list_of_momentums:  
    # Compute percentage change for each one of the momentums in the momentum list
    returns_temp = df_closing_prices.pct_change(i)
    # Unstack the returns 
    returns_temp = pd.DataFrame(returns_temp.unstack())
    name = f'{i}_m_returns'
    returns_temp.rename(columns={0: name}, inplace = True)
    # Reset the index so we can merge based on index
    returns_temp.reset_index(inplace = True)
    # Merge newly computed returns with previously created returns
    if i ==1:
        returns = returns_temp
    else:
        returns = pd.merge(returns,returns_temp,left_on=['level_0', 'time'],right_on=['level_0', 'time'], how='left', suffixes=('_original', 'right'))

# Drop nulls and set index
returns.dropna(axis=0, how='any', inplace=True)
returns.set_index(['level_0', 'time'], inplace=True)

# Generate feature data and preview first 10 rows.
X = returns
X.head(10)

,,1_m_returns,5_m_returns,10_m_returns
level_0,time,,,
FB,2021-01-06 15:00:00-05:00,-0.001960,-0.000151,0.000869
AMZN,2021-01-06 15:00:00-05:00,-0.000717,-0.000555,0.000515
AAPL,2021-01-06 15:00:00-05:00,-0.002345,-0.002267,0.004091
NFLX,2021-01-06 15:00:00-05:00,0.000593,0.000257,-0.000217
GOOGL,2021-01-06 15:00:00-05:00,0.000000,-0.000070,-0.000778
MSFT,2021-01-06 15:00:00-05:00,-0.000770,-0.001213,0.000654
TSLA,2021-01-06 15:00:00-05:00,-0.000735,-0.001587,0.010500


#### 2. Using `joblib`, load the chosen model.

In [32]:
# Load the previously trained and saved model using joblib
# YOUR CODE HERE

#### 3. Use the model file to make predicttions:
* Use `predict` on `X` and save this as `y_pred`.
* Convert `y_pred` to a DataFrame, setting the index to the index of `X`.
* Rename the column 0 to 'buy', be sure to set `inplace =True`.

In [33]:
# Use the model file to predict on X
# YOUR CODE HERE

# Convert y_pred to a dataframe, set the index to the index of X
# YOUR CODE HERE

# Rename the column 0 to 'buy', be sure to set inplace =True
# YOUR CODE HERE

,,buy
level_0,time,
FB,2021-01-06 15:00:00-05:00,1
AMZN,2021-01-06 15:00:00-05:00,1
AAPL,2021-01-06 15:00:00-05:00,1
NFLX,2021-01-06 15:00:00-05:00,0
GOOGL,2021-01-06 15:00:00-05:00,1
MSFT,2021-01-06 15:00:00-05:00,1
TSLA,2021-01-06 15:00:00-05:00,0


#### 4. Filter the stocks where 'buy' is equal to 1, saving the filter as `y_pred`.

In [34]:
# Filter the stocks where 'buy' is equal to 1
# YOUR CODE HERE

#### 5. Using the `y_pred` filter, create a dictionary called `buy_dict` and assign 'n' to each Ticker (key value) as a placeholder.

In [35]:
# Create dictionary from y_pred and assign a 'n' to each of them for now as a placeholder.
buy_dict = dict.fromkeys(y_pred.index.get_level_values(0), 'n')
buy_dict

{'FB': 'n', 'AMZN': 'n', 'AAPL': 'n', 'GOOGL': 'n', 'MSFT': 'n'}

#### 6. Obtain the total available equity in your account from the Alpaca API and store in a variable called `total_capital`. You will split the capital equally between all selected stocks per the CIO's request.

In [36]:
# Pull the total available equity in our account from the  Alpaca API
# YOUR CODE HERE

Total available capital: 99433.95


In [37]:
# Compute capital per stock, divide equity in account by number of stocks
# Use Alpaca API to pull the equity in the account
if len(buy_dict) > 0:
    capital_per_stock = float(total_capital)/ len(buy_dict)
else:
    capital_per_stock = 0
print(f'Capital per stock: {capital_per_stock}')

Capital per stock: 19886.79


#### 7. Use a for-loop to iterate through `buy_dict` to determine the number stocks you need to buy for each ticker.

In [38]:
# Use for loop to iterate through dictionary of buys 
# Determine the number stocks we need to buy for each ticker
for ticker in buy_dict:
    try:
        buy_dict[ticker] = int(capital_per_stock /int(prices[ticker].iloc[-1]['close']))
    except:
        pass

print(buy_dict)

{'FB': 75, 'AMZN': 6, 'AAPL': 156, 'GOOGL': 11, 'MSFT': 92}


#### 8. Cancel all previous orders in the Alpaca API (so you don't buy more than intended) and sell all currently held stocks to close all positions.

In [39]:
# Cancel all previous orders in the Alpaca API
# YOUR CODE HERE

# Sell all currently held stocks to close all positions
# YOUR CODE HERE

[]

#### 9. Iterate through `buy_dict` and send a buy order for each ticker with their corresponding number of shares.

In [40]:
# Iterate through the longlist object and send a buy order for each ticker with a corresponding number of shares:
# YOUR CODE HERE

buying FB numShare 75
buying AMZN numShare 6
buying AAPL numShare 156
buying GOOGL numShare 11
buying MSFT numShare 92


### Automate the algorithm

#### 1. Make a function called `trade()` that incorporates all of the steps above.

In [41]:
# Add all of the steps conducted above into the function trade
def trade():

    ticker_list = ['FB','AMZN','AAPL','NFLX', 'GOOGL', 'MSFT', 'TSLA']
    # Notice that we remove the start and end variables since we want the latest prices.
    timeframe='1Min'
    # Use iloc to get the last 10 mins every time we pull new data
    prices = api.get_barset(ticker_list, "minute").df.iloc[-11:]
    prices.ffill(inplace=True)   

    # Create and empty DataFrame for closing prices
    df_closing_prices = pd.DataFrame()

    # Fetch the closing prices of our tickers
    df_closing_prices["FB"] = prices["FB"]["close"]
    df_closing_prices["AMZN"] = prices["AMZN"]["close"]
    df_closing_prices["AAPL"] = prices["AAPL"]["close"]
    df_closing_prices["NFLX"] = prices["NFLX"]["close"]
    df_closing_prices["GOOGL"] = prices["GOOGL"]["close"]
    df_closing_prices['MSFT'] = prices['MSFT']["close"]
    df_closing_prices['TSLA'] = prices['TSLA']["close"]
    print(df_closing_prices.head())
    
    # Loop through momentums to build new DataFrame
    list_of_momentums = [1,5,10]
    for i in list_of_momentums:   
        returns_temp = df_closing_prices.pct_change(i)
        returns_temp = pd.DataFrame(returns_temp.unstack())
        name = f'{i}_m_returns'
        returns_temp.rename(columns={0: name}, inplace = True)
        returns_temp.reset_index(inplace = True)
        if i ==1:
            returns = returns_temp
        else:
            returns = pd.merge(returns,returns_temp,left_on=['level_0', 'time'],right_on=['level_0', 'time'], how='left', suffixes=('_original', 'right'))

    # Drop nulls and set index            
    returns.dropna(axis=0, how='any', inplace=True)
    returns.set_index(['level_0', 'time'], inplace=True)

    # Preprocess data for model
    # YOUR CODE HERE

    # Create the `buy_dict` object
    # YOUR CODE HERE
    
    # Split capital between stocks and determine buy or sell
    # YOUR CODE HERE

    
    # Cancel pending orders and close positions
    # YOUR CODE HERE
   
    
    # Submit orders
    # YOUR CODE HERE


#### 2. Import Python's schedule module.

In [42]:
# Import Python's schedule module 
# YOUR CODE HERE

#### 3. Use the "schedule" module to automate the algorithm:
* Clear the schedule with `.clear()`.
* Define a schedule to run the trade function every minute at 5 seconds past the minute mark (e.g. `10:31:05`).
* Use the Alpaca API to check whether the market is open.
* Use run_pending() function inside schedule to execute the schedule you defined while the market is open

In [ ]:
# Clear the schedule
# YOUR CODE HERE

# Define a schedule to run the trade function every minute at 5 seconds past the minute mark (e.g. 10:31:05)
# YOUR CODE HERE

# Use the Alpaca API to check whether the market is open
# YOUR CODE HERE

# Use run_pending() function inside schedule to execute the schedule you defined as long as the market is open
# YOUR CODE HERE

                               FB     AMZN     AAPL     NFLX    GOOGL  \
time                                                                    
2021-02-02 15:13:00-05:00  267.24  3401.17  134.705  552.420  1924.72   
2021-02-02 15:14:00-05:00  267.42  3397.59  134.665  552.530  1924.84   
2021-02-02 15:15:00-05:00  267.30  3395.85  134.700  552.690  1924.96   
2021-02-02 15:16:00-05:00  267.11  3393.24  134.730  552.320  1925.13   
2021-02-02 15:17:00-05:00  267.19  3397.00  134.820  552.685  1926.48   

                              MSFT    TSLA  
time                                        
2021-02-02 15:13:00-05:00  240.705  878.60  
2021-02-02 15:14:00-05:00  240.590  878.05  
2021-02-02 15:15:00-05:00  240.640  878.22  
2021-02-02 15:16:00-05:00  240.460  878.75  
2021-02-02 15:17:00-05:00  240.670  878.75  
buying AAPL numShare 371
buying TSLA numShare 56
                               FB      AMZN     AAPL     NFLX    GOOGL  \
time                                              